In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platforms", 'cpu')


from ImageD11.parameters import AnalysisSchema
from ImageD11.columnfile import columnfile

In [2]:
pars = AnalysisSchema.from_default().geometry_pars_obj

In [3]:
pars.set('tilt_x', 0.00123)
pars.set('tilt_y', -0.0345)
pars.set('tilt_z', 0.02)
pars.set('chi', 1)
pars.set('wedge', -3)

In [4]:
nrows = 100_000

fc = np.random.random(nrows) * 2048
sc = np.random.random(nrows) * 2048
om = np.random.random(nrows) * 360

In [5]:
cf = columnfile(new=True)
cf.nrows = nrows

cf.addcolumn(fc, 'fc')
cf.addcolumn(sc, 'sc')
cf.addcolumn(om, 'omega')

In [6]:
cf.parameters = pars

In [7]:
cf.updateGeometry()

In [8]:
cf.titles

['fc', 'sc', 'omega', 'xl', 'yl', 'zl', 'tth', 'eta', 'ds', 'gx', 'gy', 'gz']

In [9]:
gvecs = np.column_stack([cf.gx, cf.gy, cf.gz])

In [10]:
import ImageD11.transform, ImageD11.gv_general

In [11]:
import transform as mytrans

In [68]:
import importlib
importlib.reload(mytrans)

<module 'transform' from '/home/esrf/james1997a/Code/Anri/anri/sandbox/transform.py'>

In [13]:
%%time

xyz_id11 = ImageD11.transform.compute_xyz_lab((cf.sc, cf.fc), **pars.parameters)

CPU times: user 590 ms, sys: 30.5 ms, total: 620 ms
Wall time: 7.62 ms


In [14]:
%%time

xyz_me = mytrans.xyz_lab_from_sc_fc(cf.sc, cf.fc,
                                              pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                              pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                              pars.get('tilt_x'),
                                              pars.get('distance'),
                                              pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22')
                                             )

CPU times: user 12.1 s, sys: 319 ms, total: 12.4 s
Wall time: 227 ms


In [15]:
assert np.allclose(xyz_id11, xyz_me.T)

In [16]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [17]:
t_me = mytrans.lab_to_sample(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [18]:
t_id11.shape

(3, 100000)

In [19]:
t_me.shape

(100000, 3)

In [20]:
assert np.allclose(t_id11, t_me.T)

In [21]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 547 ms, sys: 31.3 ms, total: 578 ms
Wall time: 34.3 ms


In [22]:
tth_id11

array([20.90966028, 12.90344738, 20.04392288, ..., 12.16225556,
       14.86039596, 14.66895017])

In [23]:
%%time

tth_me, (eta1_me, eta2_me), (omega1_me, omega2_me) = mytrans.tth_eta_omega_from_g(gvecs, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 9.07 s, sys: 172 ms, total: 9.25 s
Wall time: 312 ms


In [24]:
tth_me

Array([20.90966028, 12.90344738, 20.04392288, ..., 12.16225556,
       14.86039596, 14.66895017], dtype=float64)

In [25]:
assert np.allclose(tth_id11, tth_me)
assert np.allclose(eta1_id11, eta1_me)
assert np.allclose(eta2_id11, eta2_me)
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)

In [26]:
eta1_me

Array([172.80252499,  84.58086535,  20.94851858, ...,  10.31620241,
        30.96771396,  75.34135546], dtype=float64)

In [27]:
eta1_id11

array([172.80252499,  84.58086535,  20.94851858, ...,  10.31620241,
        30.96771396,  75.34135546])

In [28]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))

In [29]:
k_id11

array([[-0.23142142, -0.08873811, -0.21284662, ..., -0.07887364,
        -0.11753301, -0.11454063],
       [-0.15713252, -0.78121506,  0.39104071, ..., -0.13258168,
        -0.46373486,  0.85251994],
       [-1.24427294,  0.0741098 ,  1.13916586, ...,  0.72837895,
         0.77277045,  0.25514215]])

In [30]:
k_me = mytrans.k_from_tth_eta(cf.tth, cf.eta, pars.get('wavelength'))

In [31]:
k_me

Array([[-0.23142142, -0.15713252, -1.24427294],
       [-0.08873811, -0.78121506,  0.0741098 ],
       [-0.21284662,  0.39104071,  1.13916586],
       ...,
       [-0.07887364, -0.13258168,  0.72837895],
       [-0.11753301, -0.46373486,  0.77277045],
       [-0.11454063,  0.85251994,  0.25514215]], dtype=float64)

In [32]:
k_id11.shape

(3, 100000)

In [33]:
k_me.shape

(100000, 3)

In [34]:
assert np.allclose(k_id11.T, k_me)

In [74]:
# test computation of angles for g-vectors

W =  mytrans.wmat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))
pre = (C @ W).T

k_id11 = ImageD11.gv_general.g_to_k(gvecs.T, pars.get('wavelength'), axis=[0,0,1], pre=pre, post=None)

In [79]:
k_id11

(array([ -89.02955417,   90.94665631,  112.14395828, ...,  -43.60232194,
        -170.52170011,   17.49907343]),
 array([  90.97044583,  -75.99273904,  -13.4744276 , ..., -164.32896333,
          36.44759436, -147.13237627]),
 array([False,  True,  True, ...,  True,  True,  True]))

In [80]:
k_me = mytrans.omega_solns_for_g(gvecs, pars.get('wavelength'), np.array([0,0,1]), pre, jnp.eye(3))

In [81]:
k_me

(Array([ -89.02955417,   90.94665631,  112.14395828, ...,  -43.60232194,
        -170.52170011,   17.49907343], dtype=float64),
 Array([  90.97044583,  -75.99273904,  -13.4744276 , ..., -164.32896333,
          36.44759436, -147.13237627], dtype=float64),
 Array([False,  True,  True, ...,  True,  True,  True], dtype=bool))

In [83]:
assert np.allclose(k_id11[0], k_me[0])
assert np.allclose(k_id11[1], k_me[1])
assert np.allclose(k_id11[2], k_me[2])

In [35]:
# now use k vectors to test g_from_k

In [36]:
g_id11 = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [37]:
g_id11

array([[ 2.43390149e-01, -7.78945867e-01, -3.62299814e-01, ...,
         3.80168168e-04,  1.56790470e-01, -3.77969006e-01],
       [-1.88415148e-02, -1.00100379e-01, -3.33831450e-01, ...,
         1.67474005e-01, -4.50661337e-01,  7.79388356e-01],
       [-1.25174593e+00,  8.29875609e-02,  1.11946898e+00, ...,
         7.25456516e-01,  7.73536885e-01,  2.33881473e-01]])

In [39]:
g_me = mytrans.g_from_k_omega(k_me, cf.omega, pars.get('wedge'), pars.get('chi'))

In [40]:
g_me

Array([[ 2.43390149e-01, -1.88415148e-02, -1.25174593e+00],
       [-7.78945867e-01, -1.00100379e-01,  8.29875609e-02],
       [-3.62299814e-01, -3.33831450e-01,  1.11946898e+00],
       ...,
       [ 3.80168168e-04,  1.67474005e-01,  7.25456516e-01],
       [ 1.56790470e-01, -4.50661337e-01,  7.73536885e-01],
       [-3.77969006e-01,  7.79388356e-01,  2.33881473e-01]],      dtype=float64)

In [41]:
assert np.allclose(g_id11.T, g_me)

In [59]:
g_id11_trans = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [60]:
g_id11_trans

array([[ 2.43390149e-01, -7.78945867e-01, -3.62299814e-01, ...,
         3.80168168e-04,  1.56790470e-01, -3.77969006e-01],
       [-1.88415148e-02, -1.00100379e-01, -3.33831450e-01, ...,
         1.67474005e-01, -4.50661337e-01,  7.79388356e-01],
       [-1.25174593e+00,  8.29875609e-02,  1.11946898e+00, ...,
         7.25456516e-01,  7.73536885e-01,  2.33881473e-01]])

In [61]:
# C @ W -Z works

W =  mytrans.wmat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))

post = C @ W

g_id11_gvgeneral = ImageD11.gv_general.k_to_g(k_id11, cf.omega, axis=np.array([0., 0., -1]), pre=None, post=post)

In [62]:
g_id11_gvgeneral

array([[ 2.43390149e-01, -7.78945867e-01, -3.62299814e-01, ...,
         3.80168168e-04,  1.56790470e-01, -3.77969006e-01],
       [-1.88415148e-02, -1.00100379e-01, -3.33831450e-01, ...,
         1.67474005e-01, -4.50661337e-01,  7.79388356e-01],
       [-1.25174593e+00,  8.29875609e-02,  1.11946898e+00, ...,
         7.25456516e-01,  7.73536885e-01,  2.33881473e-01]])

In [63]:
assert np.allclose(g_id11_trans, g_id11_gvgeneral)

In [47]:
g_id11 = ImageD11.transform.compute_g_vectors(cf.tth, cf.eta, cf.omega, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [48]:
g_id11

array([[ 2.43390149e-01, -7.78945867e-01, -3.62299814e-01, ...,
         3.80168168e-04,  1.56790470e-01, -3.77969006e-01],
       [-1.88415148e-02, -1.00100379e-01, -3.33831450e-01, ...,
         1.67474005e-01, -4.50661337e-01,  7.79388356e-01],
       [-1.25174593e+00,  8.29875609e-02,  1.11946898e+00, ...,
         7.25456516e-01,  7.73536885e-01,  2.33881473e-01]])

In [49]:
assert np.allclose(gvecs, g_id11.T)

In [53]:
g_me = mytrans.g_from_tth_eta_omega(cf.tth, cf.eta, cf.omega, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [54]:
g_me

Array([[ 2.43390149e-01, -1.88415148e-02, -1.25174593e+00],
       [-7.78945867e-01, -1.00100379e-01,  8.29875609e-02],
       [-3.62299814e-01, -3.33831450e-01,  1.11946898e+00],
       ...,
       [ 3.80168168e-04,  1.67474005e-01,  7.25456516e-01],
       [ 1.56790470e-01, -4.50661337e-01,  7.73536885e-01],
       [-3.77969006e-01,  7.79388356e-01,  2.33881473e-01]],      dtype=float64)

In [55]:
assert np.allclose(gvecs, g_me)

In [56]:
assert np.allclose(gvecs, mytrans.sample_to_lab(mytrans.lab_to_sample(gvecs, cf.omega, pars.get('wedge'), pars.get('chi')), cf.omega, pars.get('wedge'), pars.get('chi')))

In [57]:
# so if we can go from g to k
# why do we need this big scary function
# it's for unknown omega